In [29]:
import torch.nn as nn
import os
import psutil
import pickle
import torch
import warnings
import wandb
import time
import numpy as np
import pandas as pd
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping


from src.models import EGAT, EGRAPHSAGE
# from src.models import EGAT, EGCN, EGRAPHSAGE
from src.lightning_model import GraphModel
from src.lightning_data import GraphDataModule
from src.dataset.dataset_info import datasets
from local_variables import local_datasets_path

warnings.filterwarnings("ignore", ".*does not have many workers.*")


In [30]:
using_wandb = False
save_top_k = 1

In [31]:
my_datasets = [
    datasets["cic_ton_iot"],
    datasets["cic_ids_2017"],
    datasets["cic_ton_iot_modified"],
    datasets["ccd_inid_modified"],
    datasets["nf_uq_nids_modified"],
    datasets["edge_iiot"],
    datasets["nf_cse_cic_ids2018"],
    datasets["nf_uq_nids"],
    datasets["x_iiot"],
]

In [32]:
process = psutil.Process(os.getpid())


In [33]:
early_stopping_patience = max_epochs = 1
# early_stopping_patience = 20
learning_rate = 0.005
weight_decay = 0.0
ndim_out = [128, 128]
num_layers = 2
number_neighbors = [25, 10]
activation = F.relu
dropout = 0.0
residual = True
multi_class = True
use_centralities_nfeats = False
sort_timestamp = False
aggregation = "mean"

run_dtime = time.strftime("%Y%m%d-%H%M%S")


graph_type = "flow"
# graph_type = "window"
batch_size = 1
# graph_type = "line"

window_size = 1000

g_type = ""
if graph_type == "flow":
    g_type = "flow"
elif graph_type == "window":
    g_type = f"window_graph_{window_size}"

if multi_class:
    g_type += "__multi_class"

if use_centralities_nfeats:
    g_type += "__n_feats"

if sort_timestamp:
    g_type += "__sorted"
else:
    g_type += "__unsorted"

In [34]:
time_elapsed_dict = {}
mem_usage_dict = {}

for dataset in my_datasets:
    dataset_folder = os.path.join(local_datasets_path, dataset.name)
    graphs_folder = os.path.join(dataset_folder, g_type)

    logs_folder = os.path.join("logs", dataset.name)
    os.makedirs(logs_folder, exist_ok=True)
    wandb_runs_path = os.path.join("logs", "wandb_runs")
    os.makedirs(wandb_runs_path, exist_ok=True)

    labels_mapping = {0: "Normal", 1: "Attack"}
    num_classes = 2
    if multi_class:
        with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
            labels_names = pickle.load(f)
        labels_mapping = labels_names[0]
    num_classes = len(labels_mapping)

    dataset_kwargs = dict(
        use_node_features=use_centralities_nfeats,
        multi_class=True,
        using_masking=False,
        masked_class=2,
        num_workers=0,
        label_col=dataset.label_col,
        class_num_col=dataset.class_num_col,
        device='cuda' if torch.cuda.is_available() else "cpu"
    )

    data_module = GraphDataModule(
        graphs_folder, graph_type, batch_size=1, **dataset_kwargs)
    data_module.setup()

    ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
    edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

    my_models = {
        # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
        #               dropout, residual, num_classes),
        f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
                                                 residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
        f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
                                                             residual, num_classes, num_neighbors=None, aggregation=aggregation),
        "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
                                    residual, num_classes, num_neighbors=None),
        # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
        #                        residual, num_classes, num_neighbors=number_neighbors),
    }

    criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

    elapsed = {}
    mem_usage = {}

    for model_name, model in my_models.items():
        process = psutil.Process(os.getpid())
        peak_memory = 0

        config = {
            "run_dtime": run_dtime,
            "type": "GNN",
            "model_name": model_name,
            "max_epochs": max_epochs,
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "ndim": ndim,
            "edim": edim,
            "ndim_out": ndim_out,
            "num_layers": num_layers,
            "number_neighbors": number_neighbors,
            "activation": activation.__name__,
            "dropout": dropout,
            "residual": residual,
            "multi_class": multi_class,
            "aggregation": aggregation,
            # "details": "updating edge features",
            "early_stopping_patience": early_stopping_patience,
            "use_centralities_nfeats": use_centralities_nfeats,
        }

        graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
                                    labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True, verbose=False)

        if using_wandb:
            wandb_logger = WandbLogger(
                project=f"GNN-Analysis-{dataset.name}",
                name=model_name,
                config=config,
                save_dir=wandb_runs_path
            )
        else:
            wandb_logger = None

        f1_checkpoint_callback = ModelCheckpoint(
            monitor="val_f1_score",
            mode="max",
            filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
            save_top_k=save_top_k,
            save_on_train_epoch_end=False,
            verbose=False,
        )
        early_stopping_callback = EarlyStopping(
            monitor="val_loss",
            mode="min",
            patience=early_stopping_patience,
            verbose=False,
        )

        trainer = pl.Trainer(
            max_epochs=max_epochs,
            num_sanity_val_steps=0,
            # log_every_n_steps=0,
            callbacks=[
                f1_checkpoint_callback,
                early_stopping_callback
            ],
            default_root_dir=logs_folder,
            logger=wandb_logger,
        )

        trainer.fit(graph_model, datamodule=data_module)

        test_results = []
        test_elapsed = []
        print(
            f"==>> f1_checkpoint_callback.best_k_models.keys(): {f1_checkpoint_callback.best_k_models.keys()}")
        for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
            graph_model.test_prefix = f"best_f1_{i}"
            results = trainer.test(
                graph_model, datamodule=data_module, ckpt_path=k, verbose=False)
            mem_now = process.memory_info().rss / 1024 / 1024  # in MB
            peak_memory = max(peak_memory, mem_now)
            test_results.append(results[0][f"best_f1_{i}_test_f1"])
            test_elapsed.append(results[0][f"best_f1_{i}_elapsed"])

        logs = {
            "median_f1_of_best_f1": np.median(test_results),
            "max_f1_of_best_f1": np.max(test_results),
            "avg_f1_of_best_f1": np.mean(test_results)
        }
        elapsed[model_name] = np.mean(test_elapsed).item()
        mem_usage[model_name] = peak_memory
        print(f"==>> model_name: {model_name}")
        print(f"==>> test_elapsed: {np.mean(test_elapsed)}")
        if using_wandb:
            wandb.log(logs)
            wandb.finish()
        else:
            trainer.logger.log_metrics(logs, step=trainer.global_step)

    time_elapsed_dict[dataset.name] = elapsed
    

    mem_usage_dict[dataset.name] = mem_usage

print(f"==>> time_elapsed_dict: {time_elapsed_dict}")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | EGRAPHSAGE       | 33.8 K | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
33.8 K    Trainable params
0         Non-trainable params
33.8 K    Total params
0.135     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode
c:\Users\Administrateur\Desktop\GNN-NIDS-Analysis-WandB\.venv\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 1/1 [00:07<00:00,  0.13it/s, v_num=15, val_loss=2.130, val_acc=15.00, val_f1_score=19.30, train_loss=2.320, train_acc=5.290, train_f1_score=8.630]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:07<00:00,  0.13it/s, v_num=15, val_loss=2.130, val_acc=15.00, val_f1_score=19.30, train_loss=2.320, train_acc=5.290, train_f1_score=8.630]
==>> f1_checkpoint_callback.best_k_models.keys(): dict_keys(['logs\\cic_ton_iot\\lightning_logs\\version_15\\checkpoints\\best-val-f1-epoch=00-val_f1_score=19.26.ckpt'])


Restoring states from the checkpoint path at logs\cic_ton_iot\lightning_logs\version_15\checkpoints\best-val-f1-epoch=00-val_f1_score=19.26.ckpt
Loaded model weights from the checkpoint at logs\cic_ton_iot\lightning_logs\version_15\checkpoints\best-val-f1-epoch=00-val_f1_score=19.26.ckpt


...

Epoch 0: 100%|██████████| 1/1 [00:04<00:00,  0.22it/s, v_num=18, val_loss=2.100, val_acc=20.10, val_f1_score=24.00, train_loss=2.330, train_acc=5.600, train_f1_score=6.710]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:04<00:00,  0.22it/s, v_num=18, val_loss=2.100, val_acc=20.10, val_f1_score=24.00, train_loss=2.330, train_acc=5.600, train_f1_score=6.710]
==>> f1_checkpoint_callback.best_k_models.keys(): dict_keys(['logs\\x_iiot\\lightning_logs\\version_18\\checkpoints\\best-val-f1-epoch=00-val_f1_score=23.96.ckpt'])


Restoring states from the checkpoint path at logs\x_iiot\lightning_logs\version_18\checkpoints\best-val-f1-epoch=00-val_f1_score=23.96.ckpt
Loaded model weights from the checkpoint at logs\x_iiot\lightning_logs\version_18\checkpoints\best-val-f1-epoch=00-val_f1_score=23.96.ckpt


Testing DataLoader 0: 100%|██████████| 1/1 [00:03<00:00,  0.26it/s]
==>> model_name: e_gat_no_sampling
==>> test_elapsed: 0.9356918931007385
==>> time_elapsed_dict: {'cic_ton_iot': {'e_graphsage_mean': 0.3457059860229492, 'e_graphsage_mean_no_sampling': 0.3931763172149658, 'e_gat_no_sampling': 3.6660964488983154}, 'cic_ids_2017': {'e_graphsage_mean': 0.17475399374961853, 'e_graphsage_mean_no_sampling': 0.20365899801254272, 'e_gat_no_sampling': 5.114368438720703}, 'cic_ton_iot_modified': {'e_graphsage_mean': 0.30604270100593567, 'e_graphsage_mean_no_sampling': 0.34326601028442383, 'e_gat_no_sampling': 3.266692638397217}, 'ccd_inid_modified': {'e_graphsage_mean': 0.012954294681549072, 'e_graphsage_mean_no_sampling': 0.021133802831172943, 'e_gat_no_sampling': 0.5526881217956543}, 'nf_uq_nids_modified': {'e_graphsage_mean': 0.16071170568466187, 'e_graphsage_mean_no_sampling': 0.1268211007118225, 'e_gat_no_sampling': 1.5403748750686646}, 'edge_iiot': {'e_graphsage_mean': 0.13440439105033875

In [35]:
# Convert the nested dictionary into a DataFrame
time_df = pd.DataFrame.from_dict(time_elapsed_dict, orient='index')
mem_df = pd.DataFrame.from_dict(mem_usage_dict, orient='index')

# Optional: rename memory columns to avoid overlap
mem_df = mem_df.add_suffix('_memory_MB')

# Merge both
df = pd.concat([time_df, mem_df], axis=1)
df

,e_graphsage_mean,e_graphsage_mean_no_sampling,e_gat_no_sampling,e_graphsage_mean_memory_MB,e_graphsage_mean_no_sampling_memory_MB,e_gat_no_sampling_memory_MB
cic_ton_iot,0.345706,0.393176,3.666096,3275.417969,2965.042969,3314.628906
cic_ids_2017,0.174754,0.203659,5.114368,2579.078125,2617.851562,2712.066406
cic_ton_iot_modified,0.306043,0.343266,3.266693,2665.382812,3010.746094,3030.972656
ccd_inid_modified,0.012954,0.021134,0.552688,1645.660156,1661.386719,1656.949219
nf_uq_nids_modified,0.160712,0.126821,1.540375,2286.519531,2566.164062,2486.046875
edge_iiot,0.134404,0.157396,0.682359,2902.039062,3033.359375,3241.714844
nf_cse_cic_ids2018,0.417993,0.474915,26.616995,2707.246094,2715.402344,2807.449219
nf_uq_nids,0.554350,0.580246,26.428926,3430.597656,3744.269531,3478.031250
x_iiot,0.046652,0.050025,0.935692,2592.039062,2601.929688,2615.734375


In [36]:
# Calculate the average time for each model (i.e.,column-wise mean)
average_times = df.mean(axis=0)
average_times

e_graphsage_mean                             0.239285
e_graphsage_mean_no_sampling                 0.261182
e_gat_no_sampling                            7.644910
e_graphsage_mean_memory_MB                2675.997830
e_graphsage_mean_no_sampling_memory_MB    2768.461372
e_gat_no_sampling_memory_MB               2815.954861
dtype: float64

In [37]:
df.to_dict()

{'e_graphsage_mean': {'cic_ton_iot': 0.3457059860229492,
  'cic_ids_2017': 0.17475399374961853,
  'cic_ton_iot_modified': 0.30604270100593567,
  'ccd_inid_modified': 0.012954294681549072,
  'nf_uq_nids_modified': 0.16071170568466187,
  'edge_iiot': 0.13440439105033875,
  'nf_cse_cic_ids2018': 0.417992502450943,
  'nf_uq_nids': 0.5543503761291504,
  'x_iiot': 0.04665200412273407},
 'e_graphsage_mean_no_sampling': {'cic_ton_iot': 0.3931763172149658,
  'cic_ids_2017': 0.20365899801254272,
  'cic_ton_iot_modified': 0.34326601028442383,
  'ccd_inid_modified': 0.021133802831172943,
  'nf_uq_nids_modified': 0.1268211007118225,
  'edge_iiot': 0.15739619731903076,
  'nf_cse_cic_ids2018': 0.47491469979286194,
  'nf_uq_nids': 0.5802456736564636,
  'x_iiot': 0.05002470314502716},
 'e_gat_no_sampling': {'cic_ton_iot': 3.6660964488983154,
  'cic_ids_2017': 5.114368438720703,
  'cic_ton_iot_modified': 3.266692638397217,
  'ccd_inid_modified': 0.5526881217956543,
  'nf_uq_nids_modified': 1.54037487506